# 1) extract eid, ethnicity feature(three times instances) from main dataset .csv into .npy
+ [UKB Data-Field 21000](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=21000)
+ Output file name is the column index value

In [1]:
import numpy as np
import os
import csv
import sys

# csv.field_size_limit(sys.maxsize)  # python3 do not need this

n_participants = 502506 # number of individuals for ukb41910 dataset,including header

ukb_char_ethnic = 9747 # 21000-0.0
ukb_relation_participants = 10007 # 22021-0.0

def save_data(dir_save, names, data):
    if not os.path.exists(dir_save): # create file save path 
        os.makedirs(dir_save) 
    if type(names) is list:  # store to be multi-files
        for i, name in enumerate(names):
            np.save(os.path.join(dir_save, str(name)), data[i])
    else:
        np.save(os.path.join(dir_save, str(names)), data)
    
def get_data(dir_file, names):
    # Get the data of item names for all individuals
    # return the data list if names is char; a list containing data lists if names is a list
    
    if type(names) is list:
        data = [[] for i in names]
    else:
        data = []
    
    inds = names # get_ind(dir_file, names)
        
    with open(dir_file, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for i, row in enumerate(reader):
            if np.mod(i, 10000) == 0:   # precessed progression  
                print('Processed data:',i)
            if type(names) is list:
                for j, ind in enumerate(inds):
                    #print(str(len(row))+'\t'+str(ind))
                    data[j].append(row[ind])
            else:
                data.append(row[inds])
    return data

def generate_data(dir_file, dir_save, names):
    names_new = []
    # only generate data which are not generated before
    for name in names:
        if not os.path.isfile(os.path.join(dir_save, str(name)) + '.npy'):
            names_new.append(name)
            
    # if all items are generated before
    if not names_new:
        return
    data_new = get_data(dir_file, names_new)
    save_data(dir_save, names_new, data_new)

# multi-columns data    
def generate_Matrixdata(dir_file, dir_save, names , filename):
    names_new = []
    # only generate data which are not generated before
    for name in names:
        if not os.path.isfile(os.path.join(dir_save, str(name)) + '.npy'):
            names_new.append(name)
            
    # if all items are generated before
    if not names_new:
        return
    data_new = get_data(dir_file, names_new)
    save_data(dir_save, filename, data_new)

In [2]:
names=[0,ukb_char_ethnic,ukb_char_ethnic+1,ukb_char_ethnic+2,ukb_relation_participants]   # column index 

dir_save = 'C:\\data\\processed'
dir_file = 'C:\\data\\ukbb\\ukb41910\\ukb41910.csv'

if False:  
    generate_data(dir_file,dir_save,names)
    
else:
    print('function off')

function off


# 2) Identify white british participants
+ ethnical background column in ukb41910 is 9747-9749
+ Using [UKB Data-Coding 1001](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=1001)
+ remove the participants who have kinship to other participants
+ [Data-Field 22021](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22021) using [Data-Coding 682](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=682)

In [3]:
if False:    
    all_person_id = np.load('C:\\data\\processed\\0.npy')
    col_ethnic_0 = np.load('C:\\data\\processed\\9747.npy')
    col_ethnic_1 = np.load('C:\\data\\processed\\9748.npy')
    col_ethnic_2 = np.load('C:\\data\\processed\\9749.npy')
    col_kinship = np.load('C:\\data\\processed\\10007.npy')
    #merge three columns and filter first column
    individual=all_person_id[1:] # [0] is header, ignore it
    ethnics=col_ethnic_0[1:]  
    col_kinship =col_kinship[1:]
    
    for i in range(n_participants-1):  #  -1 coz n_participants including header
        if ethnics[i]=='' and col_ethnic_1[i+1]!='':
            ethnics[i]=col_ethnic_1[i+1]
    for i in range(n_participants-1):
        if ethnics[i]=='' and col_ethnic_2[i+1]!='':
            ethnics[i]=col_ethnic_2[i+1]
    outfile=open('C:\\data\\processed\\extract_ind','w') 
    for i in range(n_participants-1):
        if (ethnics[i]=='1001' or ethnics[i]=='2001' or ethnics[i]=='3001' or ethnics[i]=='4001') and col_kinship[i]=='0':
            outfile.write(individual[i]+'\n')
    outfile.close()

else:
    print('function off')

function off


#### Remove withdraw participant data
> remove the withdraw data from extract_ind file  
outputFile :merge_white_Britich_clean , storing the eid of available participants.

In [4]:
if False:
    extract_ind = open('C:\\data\\processed\\extract_ind','r')
    path_withdraw = 'C:\\data\\ukbb\\ukb41910\\w60434_20210201.csv'

    withdarw_eid = []
    remain_eid = []
    with open(path_withdraw, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for i, row in enumerate(reader):   # the type of row is list
            withdarw_eid.append((int)(row[0]))  # the element of list row is str 

    for line in extract_ind:
        if (int)(line) not in withdarw_eid :  
            remain_eid.append((int)(line))  # remain eid into new list

    outfile = open('C:\\data\\processed\\merge_white_Britich_clean','w') 
    for i in range(len(remain_eid)):
        outfile.write(str(remain_eid[i])+'\n')
    outfile.close()
else:
    print('function off')

function off


# 3) Data extracting from main dataset

+ Sex using [Data-Coing 9](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=9)
+ Smoke & Alcohol using [Data-Coding 90](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=90)
+ Vascular/heart problems using [Data-Coding 100605](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=100605)

#### extract from maindata set into single column .npy file

In [60]:
# personal
ukb_char_sex = 22  # 31-0.0
ukb_char_birth = 23  # 34-0.0  year of birth
ukb_char_bmi = 9750  # 21001-0.0
ukb_char_weight = 9754  #21002
ukb_char_ageRecr = 9798  #21022 age at recruitment 
# food intake
ukb_intake_water = 758  # 1528-0.0
ukb_intake_coffee =750  # 1508-0.0

# physical activity
ukb_phys_moderate = 526 # 884-0.0
ukb_phys_vigorous = 534 # 904-0.0

# not good life-style 
ukb_smoke_status =  8843 # 20116-0.0
ukb_alcohol_status =  8847 # 20117-0.0

# relevant diseases
ukb_diag_vas = 6031 # 6150-0.0  Vascular/heart problems, 1:heart attack; 3:Stroke; 4:High Blood 

# diagnosis
ukb_diag_diabete=1049 # 2443-0.0 diabetes

# Non-cancer code self-repoted
ukb_ill_code_start = 6696 # 20002-0.0
ukb_ill_code_end = 6831 # 

# cancer code self-repoted
ukb_cancer_code_start = 6672 # 20001-0.0
ukb_cancer_code_end = 6695 # 

# date of recruitment
ukb_date_recruitment = 89 # 53-0.0

# summary
extract_col=[ukb_char_sex,ukb_char_birth,ukb_char_ageRecr,
             ukb_char_bmi,ukb_char_bmi+1,ukb_char_bmi+2,ukb_char_bmi+3,
             ukb_char_weight,ukb_char_weight+1,ukb_char_weight+2,ukb_char_weight+3,
             ukb_intake_water,ukb_intake_water+1,ukb_intake_water+2,ukb_intake_water+3,
             ukb_intake_coffee,ukb_intake_coffee+1,ukb_intake_coffee+2,ukb_intake_coffee+3,
             ukb_phys_moderate,ukb_phys_moderate+1,ukb_phys_moderate+2,ukb_phys_moderate+3,
             ukb_phys_vigorous,ukb_phys_vigorous+1,ukb_phys_vigorous+2,ukb_phys_vigorous+3,
             ukb_smoke_status,ukb_smoke_status+1,ukb_smoke_status+2,ukb_smoke_status+3,
             ukb_alcohol_status,ukb_alcohol_status+1,ukb_alcohol_status+2,ukb_alcohol_status+3,
             ukb_diag_diabete,ukb_diag_diabete+1,ukb_diag_diabete+2,ukb_diag_diabete+3,
             ukb_diag_vas,ukb_diag_vas+1,ukb_diag_vas+2,ukb_diag_vas+3,
             ukb_diag_vas+4,ukb_diag_vas+5,ukb_diag_vas+6,ukb_diag_vas+7,
             ukb_diag_vas+8,ukb_diag_vas+9,ukb_diag_vas+10,ukb_diag_vas+11,
             ukb_date_recruitment,ukb_date_recruitment+1,ukb_date_recruitment+2,ukb_date_recruitment+3
            ]

extract_disease = np.arange(ukb_cancer_code_start,ukb_ill_code_end+1,1)  # self-reported cancer/non-cancer diseases
if True: # features 
    path_data = dir_file
    path_save = 'C:\\data\\processed\\DataSplit'
    generate_data(path_data,path_save,extract_col)
    
if True: # self-reported disease data
    generate_data(path_data,path_save,extract_disease)


# Generate information of self-reported disease/cancers

+ Diabetes diagnosed by doctor   [Data-Field 2443](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=2443)
+ value using [Data-Coding 100349](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=100349)
+ Description: Has a doctor ever told you that you have diabetes

In [6]:
if False:
    # diagnosis by doctor ->medical condition 
    diabete_mainset = np.load('C:\\data\\processed\\DataSplit\\'+str(ukb_diag_diabete)+'.npy')
    diabete_mainset = diabete_mainset.reshape(len(diabete_mainset),1)
    #print(diabete_mainset)
else:
    print('not using this record')

not using this record


### Process self-reported disease/cancer information from .npy files derive from .csv main dataset 

+ Non-cancer self-reported illness information [Data-Field 20002](https://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=20002)
+ using[Data-Coding 6](https://biobank.ctsu.ox.ac.uk/crystal/coding.cgi?id=6)
+ Cancer self-reported illness information [Data-Field 20001](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=20001)
+ using[Data-Coding 3](https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=3)

> OutPUT file. These files cover all participants:
>> .data file -index is not eid. Cover all participants in main dataset,Access by index of participants in mainset   
.list file -the odd of key(disease name) is same with the rows odd of .data file

In [7]:
if False:
    from collections import defaultdict

    outdata1=open('C:\\data\\processed\\self_cancer.data','w')
    outdata2=open('C:\\\data\\processed\\self_disease.data','w')

    outlist1=open('C:\\data\\processed\\self_cancer.list','w')
    outlist2=open('C:\\data\\processed\\self_disease.list','w')

    cancer=defaultdict(list) # the value of dict is a list
    disease=defaultdict(list)
    disease_map ={'1223': 'Type2 diabetes', '1471': 'atrial fibrillation','1461':'inflammatory bowel disease','1075':'myocardial infarction'}
    cancer_map ={'1002':'breast cancer'}

    for i in range(ukb_cancer_code_start,ukb_cancer_code_end+1):

        col=np.load('C:\\data\\processed\\DataSplit\\'+str(i)+'.npy') # loop self-reported cancer columns

        for j in range(1,n_participants): 
            if col[j] in ['1002']:   # brast cancer  , 
                cancer[cancer_map[col[j]]].append(j-1)

    for i in range(ukb_ill_code_start,ukb_ill_code_end+1):

        col=np.load('C:\\data\\processed\\DataSplit\\'+str(i)+'.npy') # loop self-reported non-cancer columns

        for j in range(1,n_participants):  # index start from 1
            if col[j] in ['1223','1471','1461','1075']:   
                disease[disease_map[col[j]]].append(j-1) 
    print('finished')


    for key,value in cancer.items():
        sum_num=0
        for i in range(n_participants-1): # index start from 0
            if i in value:  # participant index in record? 
                sum_num+=1
                outdata1.write('1'+'\t')
            else:
                outdata1.write('0'+'\t')
        outdata1.write('\n')
        outlist1.write(key+'\t')
        outlist1.write(str(sum_num)+'\n')

    outdata1.close()
    outlist1.close()   


    for key,value in disease.items():
        sum_num=0
        for i in range(n_participants-1):
            if i in value:   # value list may have repeated num, but dosent matter.
                sum_num+=1
                outdata2.write('1'+'\t')
            else:
                outdata2.write('0'+'\t')
        outdata2.write('\n')
        outlist2.write(key+'\t')
        outlist2.write(str(sum_num)+'\n')

    outdata2.close()
    outlist2.close() 
else:
    print('function off')

function off


### Remove inappropriate participants from the self-reported patients

+ merge_white_Britich_clean,  obtain available participant's eid
+ 0.npy,eid->index in mainset

> inputFile: merge_white_Britich_clean , 0.npy  
ouputFile: remain_eid_index.npy ,storing the index of avaliable participants

In [8]:
if False:
    remain_eid = open('C:\\data\\processed\\merge_white_Britich_clean','r')
    mainset_eid = np.load('C:\\data\\processed\\0.npy')

    # eid mactching index of main dataset 
    individual = mainset_eid[1:].tolist() # ignore header
    ind_index = []
    count = 0
    for line in remain_eid:
        ind=line.split()[0]  # obtain remain patients eid
        ind_index.append(individual.index(ind))# remain patients eid transform to be index of mainset
        count+=1
        if np.mod(count, 10000) == 0:   # precessed progression  
            print('Processed data:',count) 
    remain_eid.close()

    # store remain_index to file remain_eid_index

    np.save('C:\\data\\processed\\remain_eid_index', np.array(ind_index)) # .npy

    eid_index = open('C:\\data\\processed\\remain_eid_index','w')   # file
    for i in ind_index:
        eid_index.write(str(i)+'\n')
    eid_index.close()
else:
    print('function off')

function off


+ Using the remaining patients' index to generate remaining self-reported cancer/disease files

In [130]:
if False:
    ind_genetics_index=np.load('C:\\data\\processed\\remain_eid_index.npy') # saving time for repeated operation

300868


In [134]:
if False:
    infile_cancer=open('C:\\data\\processed\\self_cancer.data','r')
    infile_disease=open('C:\\data\\processed\\self_disease.data','r')
    outfile_cancer=open('C:\\data\\processed\\self_cancer_sub.data','w')
    outfile_disease=open('C:\\data\\processed\\self_disease_sub.data','w')
    outfile_cancer_num=open('C:\\data\\processed\\cancer_num','w')
    outfile_disease_num=open('C:\\data\\processed\\disease_num','w')
    

    for line in infile_cancer:
        A=line.strip().split()
        sum_num=0
        for XX in ind_genetics_index:  
            outfile_cancer.write(A[XX]+'\t')
            sum_num=sum_num+int(A[XX])
        outfile_cancer_num.write(str(sum_num)+'\n')
        outfile_cancer.write('\n')

    for line in infile_disease:
        A=line.strip().split()
        sum_num=0
        for XX in ind_genetics_index:
            outfile_disease.write(A[XX]+'\t')
            sum_num=sum_num+int(A[XX])
        outfile_disease_num.write(str(sum_num)+'\n')
        outfile_disease.write('\n')
    infile_cancer.close()
    infile_disease.close()
    outfile_cancer.close()
    outfile_disease.close()
    outfile_cancer_num.close()
    outfile_disease_num.close()
else:
    print('function off')

300868


##  Process hes_diag to genereate cancer/disease information from hospitalization records

#### ICD-10 code

+ E11.X - type2 diabetes
+ I21.X,I22.X,I23.X,I24.1,I25.2 - CAD
+ I48.X - atrial fibrillation
+ K51.X - inflammatory bowel
+ C50.X - breast cancer  

Reference: Khera, A.V., Chaffin, M., Aragam, K.G. et al. Genome-wide polygenic scores for common diseases identify individuals with risk equivalent to monogenic mutations. Nat Genet 50, 1219–1224 (2018). https://doi.org/10.1038/s41588-018-0183-z
#### hes_diag have not diagnosis date ,  we capture the eid and instance index as key to match from hesin file to get date information in the following step
>OutputFile:   
1)hes_t2d_eid_ins.npy, has two columns-> T2D patients' eid & ins_index   
2)hes_CAD_eid_ins.npy  
3)hes_IBD_eid_ins.npy  
4)hes_AF_eid_ins.npy  
5)hes_BC_eid_ins.npy  
Those .npy files are already remove inapplicable participants by below preprocessed subfile

> InputFile:  
1)hesin_diag.txt  
2)merge_white_Britich_clean, flitering the inapplicable participants by subeid record

In [193]:
if False:
    import re
    from collections import defaultdict
    disease=defaultdict(list)
    hesdiag_file=open('C:\\data\\ukbb\HES\\record-level hospital inpatient data tables\\hesin_diag.txt','r')
    
    t2d_ins = []
    t2d_eid = []
    CAD_ins = []
    CAD_eid = []
    AF_ins = []
    AF_eid = []
    IBD_ins = []
    IBD_eid = []
    BC_ins = []
    BC_eid = []
    
    count=0
    # A[3] is level column ; A[6] is ICD-10 code ;A[0] is eid
    for line in hesdiag_file:
        A = line.strip('\n').split('\t') # to be a string list

        eid = A[0]
        ins_index = A[1]
        level = A[3]    
        ICD10 = A[6]
        if level == '1': 
            if re.match(r'E11*', ICD10):
                disease['type2 diabetes'].append(eid)
                t2d_eid.append(eid)
                t2d_ins.append(ins_index)
                
            if re.match(r'I21*', ICD10) or re.match(r'I22*', ICD10) or re.match(r'I23*', ICD10) or re.match(r'I241', ICD10) or re.match(r'I252', ICD10):
                disease['myocardial infarction'].append(eid)
                CAD_eid.append(eid)
                CAD_ins.append(ins_index)
            
            if re.match(r'I48*', ICD10):
                disease['atrial fibrillation'].append(eid)
                AF_eid.append(eid)
                AF_ins.append(ins_index)
                
            if re.match(r'K51*', ICD10):
                disease['inflammatory bowel disease'].append(eid)
                IBD_eid.append(eid)
                IBD_ins.append(ins_index)
                
            if re.match(r'C50*', ICD10):
                disease['breast cancer'].append(eid)
                BC_eid.append(eid)
                BC_ins.append(ins_index)
                
        count+=1
        if np.mod(count, 100000) == 0: 
            print('Process:',count)

    hesdiag_file.close()
    print('finished disease_dic')

# for macthing diagnosed date
    # load subeid file
    remain_eid = open('C:\\data\\processed\\merge_white_Britich_clean','r')
    sub_eid=[]
    for line in remain_eid:
        A=line.split()[0]
        sub_eid.append(A)
    remain_eid.close()
    
    remain_t2d_eid=[]
    remain_t2d_ins=[]
    for i in range(len(t2d_eid)):
        if t2d_eid[i] in sub_eid:
            remain_t2d_eid.append(t2d_eid[i])
            remain_t2d_ins.append(t2d_ins[i])
    remain_t2d_eid = np.array(remain_t2d_eid)
    remain_t2d_ins = np.array(remain_t2d_ins)
    t2d_eid_ins=np.concatenate((remain_t2d_eid.reshape(len(remain_t2d_eid),1),remain_t2d_ins.reshape(len(remain_t2d_ins),1)),axis =1)
    np.save('C:\\data\\processed\\hes_t2d_eid_ins', t2d_eid_ins)
    print('finished saving HES-T2D patient eid and ins_index')
    remain_CAD_eid=[]
    remain_CAD_ins=[]
    for i in range(len(CAD_eid)):
        if CAD_eid[i] in sub_eid:
            remain_CAD_eid.append(CAD_eid[i])
            remain_CAD_ins.append(CAD_ins[i])
    remain_CAD_eid = np.array(remain_CAD_eid)
    remain_CAD_ins = np.array(remain_CAD_ins)
    CAD_eid_ins=np.concatenate((remain_CAD_eid.reshape(len(remain_CAD_eid),1),remain_CAD_ins.reshape(len(remain_CAD_ins),1)),axis =1)
    np.save('C:\\data\\processed\\hes_CAD_eid_ins', CAD_eid_ins)
    print('finished saving HES-CAD patient eid and ins_index')
    remain_AF_eid=[]
    remain_AF_ins=[]
    for i in range(len(AF_eid)):
        if AF_eid[i] in sub_eid:
            remain_AF_eid.append(AF_eid[i])
            remain_AF_ins.append(AF_ins[i])
    remain_AF_eid = np.array(remain_AF_eid)
    remain_AF_ins = np.array(remain_AF_ins)
    AF_eid_ins=np.concatenate((remain_AF_eid.reshape(len(remain_AF_eid),1),remain_AF_ins.reshape(len(remain_AF_ins),1)),axis =1)
    np.save('C:\\data\\processed\\hes_AF_eid_ins', AF_eid_ins)
    print('finished saving HES-AF patient eid and ins_index')
    remain_IBD_eid=[]
    remain_IBD_ins=[]
    for i in range(len(IBD_eid)):
        if IBD_eid[i] in sub_eid:
            remain_IBD_eid.append(IBD_eid[i])
            remain_IBD_ins.append(IBD_ins[i])
    remain_IBD_eid = np.array(remain_IBD_eid)
    remain_IBD_ins = np.array(remain_IBD_ins)
    IBD_eid_ins=np.concatenate((remain_IBD_eid.reshape(len(remain_IBD_eid),1),remain_IBD_ins.reshape(len(remain_IBD_ins),1)),axis =1)
    np.save('C:\\data\\processed\\hes_IBD_eid_ins', IBD_eid_ins)
    print('finished saving HES-IBD patient eid and ins_index')
    remain_BC_eid=[]
    remain_BC_ins=[]
    for i in range(len(BC_eid)):
        if BC_eid[i] in sub_eid:
            remain_BC_eid.append(BC_eid[i])
            remain_BC_ins.append(BC_ins[i])
    remain_BC_eid = np.array(remain_BC_eid)
    remain_BC_ins = np.array(remain_BC_ins)
    BC_eid_ins=np.concatenate((remain_BC_eid.reshape(len(remain_BC_eid),1),remain_BC_ins.reshape(len(remain_BC_ins),1)),axis =1)
    np.save('C:\\data\\processed\\hes_BC_eid_ins', BC_eid_ins)
    print('finished saving HES-BC patient eid and ins_index')
    
#  Generate one-hot format diseases diagonsis information for available participants by HES_diag   
    outdata1=open('C:\\data\\processed\\ICD10_HES.data','w')
    outlist1=open('C:\\data\\processed\\ICD10_HES.list','w')
    mainset_eid = np.load('C:\\data\\processed\\0.npy') 
    
    eid = mainset_eid[1:]
    for key,value in disease.items():
        sum_num=0
        print(key)
        for i in range(n_participants-1): # index start from 0
            if eid[i] in value:  
                sum_num+=1
                outdata1.write('1'+'\t')
            else:
                outdata1.write('0'+'\t')
            if np.mod(i, 10000) == 0: 
                print('Process: ',i)
        outdata1.write('\n')
        outlist1.write(key+'\t')
        outlist1.write(str(sum_num)+'\n')

    outdata1.close()
    outlist1.close() 

else:
    print('function off')

function off


+ Using the remaining patients' index to generate remaining hes ICD10 diagnosis files
> Output file:
>> ICD10_HES_sub.data, every row represents one disease from three digits ICD10 code main diagnosis   
ICD10_HES_num, remaining avaliable inpatients diagnosis information

In [12]:
if False:
    sub_index=np.load('C:\\data\\processed\\remain_eid_index.npy') # saving time for repeated operation 

In [13]:
if False:
    infile_disease=open('C:\\data\\processed\\ICD10_HES.data','r')
    outfile_disease=open('C:\\data\\processed\\ICD10_HES_sub.data','w')
    outfile_disease_num=open('C:\\data\\processed\\ICD10_HES_num','w')

    for line in infile_disease:
        A=line.strip().split()
        sum_num=0
        for XX in sub_index:
            outfile_disease.write(A[XX]+'\t')
            sum_num=sum_num+int(A[XX])
        outfile_disease_num.write(str(sum_num)+'\n')
        outfile_disease.write('\n')
        
    infile_disease.close()
    outfile_disease.close()
    outfile_disease_num.close()
else:
    print('function off')

function off


## Integrate Hes_diag & self-reported diag information to generate Testing cases

+ Patients who not be diagnosed at recruitment, but be diagnosed in inpatient record
+ Using pandas date format(Timestamp) for time comparasion, ignoring those inpatient record before recruitment

>OutputFiles:  
1) eid_testCAD.npy ,the eid of patients those as testing set  
2) eid_testT2D.npy  
3) eid_testAF.npy  
4) eid_testIBD.npy  
5) eid_testBC.npy

In [219]:
if False:
    import pandas as pd

    col_recruiment_0 = np.load('C:\\data\\processed\\DataSplit\\89.npy')
    col_recruiment_1 = np.load('C:\\data\\processed\\DataSplit\\90.npy')
    col_recruiment_2 = np.load('C:\\data\\processed\\DataSplit\\91.npy')
    col_recruiment_3 = np.load('C:\\data\\processed\\DataSplit\\92.npy')
    subset_index = np.load('C:\\data\\processed\\remain_eid_index.npy')

    recruiment_date = col_recruiment_0[1:]
    sub_recruiment_date=[]
    for i in range(n_participants-1): 
        if recruiment_date[i]=='' and col_recruiment_1[i+1]!='':
            recruiment_date[i]=col_recruiment_1[i+1]
    for i in range(n_participants-1):
        if recruiment_date[i]=='' and col_recruiment_2[i+1]!='':
            recruiment_date[i]=col_recruiment_2[i+1]
    for i in range(n_participants-1):
        if recruiment_date[i]=='' and col_recruiment_3[i+1]!='':
            recruiment_date[i]=col_recruiment_3[i+1]
    for i in subset_index:  # remove the not applicable participants
        sub_recruiment_date.append(recruiment_date[i])
    sub_recruiment_date = pd.to_datetime(np.array(sub_recruiment_date), format='%Y-%m-%d') # date format transformation

    # load raw hesin file & transform epistart date format
    df=pd.read_table('C:\\data\\ukbb\HES\\record-level hospital inpatient data tables\\hesin.txt',usecols=[0,1,4])
    df['epistart']=pd.to_datetime(df['epistart'], format='%Y%m%d')
    # load preprocessed file
    hes_t2d = np.load('C:\\data\\processed\\hes_t2d_eid_ins.npy')
    hes_CAD = np.load('C:\\data\\processed\\hes_CAD_eid_ins.npy')
    hes_AF = np.load('C:\\data\\processed\\hes_AF_eid_ins.npy')
    hes_IBD = np.load('C:\\data\\processed\\hes_IBD_eid_ins.npy')
    hes_BC = np.load('C:\\data\\processed\\hes_BC_eid_ins.npy')

In [220]:
if False:
    from collections import defaultdict
    disease=defaultdict(list)
    infile1=open('C:\\data\\processed\\self_cancer_sub.data','r')
    infile2=open('C:\\data\\processed\\self_disease_sub.data','r')
    for line in infile1:
        breast_cancer=line.strip().split('\t') # do not strip'\n', it will add one null element

    disease_index = 0
    for line in infile2:
        disease[disease_index]=line.strip().split('\t')
        disease_index+=1

    infile1.close()
    infile2.close()

    # load subeid file
    remain_eid = open('C:\\data\\processed\\merge_white_Britich_clean','r')
    sub_eid=[]
    for line in remain_eid:
        A=line.split()[0]
        sub_eid.append(A)
    remain_eid.close()

+ disease dict contain four target diseases, the index of that:
> myocardial infarction: disease[0]  
atrial fibrillation:disease[2]   
inflammatory bowel disease:disease[3]  
type2 diabetes:disease[1] 

In [218]:
if False:
    testing_CAD = []
    for i in range(len(hes_CAD)):
        # format sorting 
        eid = (int)(hes_CAD[i][0])  # column 0 represent inpatient eid
        ins_index = (int)(hes_CAD[i][1]) # column 1 represent inpatient instance index
        ir_subset_index = sub_eid.index((str)(eid))

        Select = df[(df['eid']==eid)&(df['ins_index']==ins_index)] # composite key as Pk searching in hesin table
        if (Select['epistart']>sub_recruiment_date[ir_subset_index]).values[0]: # dataframe -> numpy array-> element   
            if disease[0][ir_subset_index]=='0': # self-reported(at recruitment) is not be diagnosed
                testing_CAD.append(eid)
        if np.mod(i, 10000) == 0: 
            print('Process: ',i)
        testing_CAD = list(set(testing_CAD))
    print('Finished CAD ')

    testing_t2d = []
    for i in range(len(hes_t2d)):
        # format sorting 
        eid = (int)(hes_t2d[i][0])  # column 0 represent inpatient eid
        ins_index = (int)(hes_t2d[i][1]) # column 1 represent inpatient instance index
        ir_subset_index = sub_eid.index((str)(eid))

        Select = df[(df['eid']==eid)&(df['ins_index']==ins_index)] # composite key as Pk searching in hesin table
        if (Select['epistart']>sub_recruiment_date[ir_subset_index]).values[0]: # dataframe -> numpy array-> element   
            if disease[1][ir_subset_index]=='0': # self-reported(at recruitment) is not be diagnosed
                testing_t2d.append(eid)
        if np.mod(i, 10000) == 0: 
            print('Process: ',i)
        testing_t2d = list(set(testing_t2d))
    print('Finished t2d ')

    testing_AF = []
    for i in range(len(hes_AF)):
        # format sorting 
        eid = (int)(hes_AF[i][0])  # column 0 represent inpatient eid
        ins_index = (int)(hes_AF[i][1]) # column 1 represent inpatient instance index
        ir_subset_index = sub_eid.index((str)(eid))

        Select = df[(df['eid']==eid)&(df['ins_index']==ins_index)] # composite key as Pk searching in hesin table
        if (Select['epistart']>sub_recruiment_date[ir_subset_index]).values[0]: # dataframe -> numpy array-> element   
            if disease[2][ir_subset_index]=='0': # self-reported(at recruitment) is not be diagnosed
                testing_AF.append(eid)
        if np.mod(i, 10000) == 0: 
            print('Process: ',i)
        testing_AF = list(set(testing_AF))
    print('Finished AF ')

    testing_IBD = []
    for i in range(len(hes_IBD)):
        # format sorting 
        eid = (int)(hes_IBD[i][0])  # column 0 represent inpatient eid
        ins_index = (int)(hes_IBD[i][1]) # column 1 represent inpatient instance index
        ir_subset_index = sub_eid.index((str)(eid))

        Select = df[(df['eid']==eid)&(df['ins_index']==ins_index)] # composite key as Pk searching in hesin table
        if (Select['epistart']>sub_recruiment_date[ir_subset_index]).values[0]: # dataframe -> numpy array-> element   
            if disease[3][ir_subset_index]=='0': # self-reported(at recruitment) is not be diagnosed
                testing_IBD.append(eid)
        if np.mod(i, 10000) == 0: 
            print('Process: ',i)
        testing_IBD = list(set(testing_IBD))
    print('Finished IBD ')

    testing_BC = []
    for i in range(len(hes_BC)):
        # format sorting 
        eid = (int)(hes_BC[i][0])  # column 0 represent inpatient eid
        ins_index = (int)(hes_BC[i][1]) # column 1 represent inpatient instance index
        ir_subset_index = sub_eid.index((str)(eid))

        Select = df[(df['eid']==eid)&(df['ins_index']==ins_index)] # composite key as Pk searching in hesin table
        if (Select['epistart']>sub_recruiment_date[ir_subset_index]).values[0]: # dataframe -> numpy array-> element   
            if breast_cancer[ir_subset_index]=='0': # self-reported(at recruitment) is not be diagnosed
                testing_BC.append(eid)
        if np.mod(i, 10000) == 0: 
            print('Process: ',i)
        testing_BC = list(set(testing_BC))
    print('Finished BC ')

    save_CAD = np.array(testing_CAD)
    save_CAD.sort()
    save_t2d = np.array(testing_t2d)
    save_t2d.sort()
    save_AF = np.array(testing_AF)
    save_AF.sort()
    save_IBD = np.array(testing_IBD)
    save_IBD.sort()
    save_BC = np.array(testing_BC)
    save_BC.sort()
    save_data('C:\\data\\processed\\testset','eid_testCAD', save_CAD)
    save_data('C:\\data\\processed\\testset','eid_testT2D', save_t2d)
    save_data('C:\\data\\processed\\testset','eid_testAF', save_AF)
    save_data('C:\\data\\processed\\testset','eid_testIBD', save_IBD)
    save_data('C:\\data\\processed\\testset','eid_testBC', save_BC)
    print('all done')

else:
    print('function off')

function off


### Generate the information of testing cases
> OutputFiles:  
1) eid_test_cases.npy :the eid of patients who are regarded as testing set  
2) test_cases.list : the number of target diseases cases in testing set 

In [256]:
if False:
    CAD=np.load('C:\\data\\processed\\testset\\eid_testCAD.npy')
    T2D=np.load('C:\\data\\processed\\testset\\eid_testT2D.npy')
    AF=np.load('C:\\data\\processed\\testset\\eid_testAF.npy')
    IBD=np.load('C:\\data\\processed\\testset\\eid_testIBD.npy')
    BC=np.load('C:\\data\\processed\\testset\\eid_testBC.npy')
    temp=np.concatenate((CAD,T2D,AF,IBD,BC),axis=0)
    temp=list(set(temp)) # remove repeated eid
    count_dict={'myocardial infarction':len(CAD),'type2 diabetes':len(T2D),'atrial fibrillation':len(AF),'inflammatory bowel disease':len(IBD),'breast cancer':len(BC),'n_test pool':len(temp)}
    temp.sort()
    save_data('C:\\data\\processed\\testset','eid_test_cases', temp)
    outfile =open('C:\\data\\processed\\testset\\test_cases.list','w')
    for key,value in count_dict.items():
        outfile.write(key+'\t'+str(value))
        outfile.write('\n')
    outfile.close()
else:
    print('function off')

## Integrate self-reported diseases and ICD10-codes to generate all diag information

+ Load five diseases diagnosis information generated before
> Inputfile: 1)self_cancer_sub.data,  2)self_diseas_sub.data,  3)ICD10_HES_sub.data  
Outputfile: 1) Integrated_diagnosis.data 2)Integrated_diagnosis.list

In [16]:
if False:
    from collections import defaultdict
    HES=defaultdict(list)
    disease=defaultdict(list)
    infile1=open('C:\\data\\processed\\self_cancer_sub.data','r')
    infile2=open('C:\\data\\processed\\self_disease_sub.data','r')
    infile3=open('C:\\data\\processed\\ICD10_HES_sub.data','r')

    for line in infile1:
        breast_cancer=line.strip().split('\t')

    disease_index = 0
    for line in infile2:
        disease[disease_index]=line.strip('\n').split('\t')
        disease_index+=1

    HES_index = 0
    for line in infile3:
        HES[HES_index]=line.strip('\n').split('\t')  # load HES_sub data ,HES[0] is 'myocardial infarction'
        HES_index+=1

    infile1.close()
    infile2.close()
    infile3.close()
else:
    print('function off')

function off


> Integrate HES and self-reported information
>> myocardial infarction: HES[0] -> disease[0]  
atrial fibrillation: HES[1] -> disease[2]  
breast cancer:HES[2] -> breast_cancer  
inflammatory bowel disease:HES[3] -> disease[3]  
type2 diabetes:HES[4] -> disease[1] 

In [17]:
if False:
    outdata1=open('C:\\data\\processed\\Integrated_diagnosis.data','w')
    outlist1=open('C:\\data\\processed\\Integrated_diagnosis.list','w')

    for key,value in HES.items():
        print(key)
        sum_num = 0                 

        for j in range(len(breast_cancer)): # avaliable record num - 453663
            if key==0:
                condition = ( HES[key][j] == '1' or disease[0][j]=='1')
            elif key==1:
                condition = ( HES[key][j] == '1' or disease[2][j]=='1')
            elif key==2:
                condition = ( HES[key][j] == '1' or breast_cancer[j]=='1')
            elif key==3:
                condition = ( HES[key][j] == '1' or disease[3][j]=='1')
            elif key==4:
                condition = ( HES[key][j] == '1' or disease[1][j]=='1')

            if condition:
                sum_num+=1
                outdata1.write('1'+'\t')
            else:
                outdata1.write('0'+'\t')

            if np.mod(j, 100000) == 0: 
                print('Process: ',j)
        outdata1.write('\n')
        outlist1.write(str(sum_num)+'\n')

    outdata1.close()
    outlist1.close()
else:
    print('function off')

function off


## Generate controls pool&cases set without test cases

> Outputfile:
>>1)control_pool: the eid of all controls  
2)control_pool_num: the number of controls

In [273]:
if True:
    from collections import defaultdict
    disease=defaultdict(list)
    infile=open('C:\\data\\processed\\Integrated_diagnosis.data','r')
    infile2=open('C:\\data\\processed\\merge_white_Britich_clean','r')
    test_pool=np.load('C:\\data\\processed\\testset\\eid_test_cases.npy')
    index_to_eid=[]
    disease_index=0
    
    for line in infile2:
        A=line.strip().split('\t')[0]
        sub_eid.append(A)
    infile2.close()
    for line in infile: 
        A=line.strip().rsplit()
        disease[disease_index]=A
        disease_index+=1
    infile.close()
    
    outfile1=open('C:\\data\\processed\\control_pool','w')
    outfile2=open('C:\\data\\processed\\control_pool_num','w')
    control_index=[]
    case_flag=False
    for i in range(len(disease[0])): 

        for j in range(5): # diagnosed anyone of those five diseases
            if disease[j][i]=='1':
                case_flag=True
                

        if not case_flag: # controls 
            if (int)(sub_eid[i]) not in test_pool:
                control_index.append(i)

        case_flag=False 

    n_control_pool = len(control_index)
    outfile2.write(str(n_control_pool)+'\n')
    for i in control_index:
        outfile1.write(sub_eid[i]+'\n')

    outfile1.close()
    outfile2.close()
else:
    print('function off')